In [30]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

device = torch.device('cpu') # gpu 설정

In [31]:
num_epoches = 15 
num_classes = 1
batch_size = 128 
learning_rate = 0.001

In [32]:
#Loading MNIST data set
train_dataset = torchvision.datasets.MNIST(root='../../data/', train=True, transform=transforms.ToTensor(),download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data/', train=False, transform=transforms.ToTensor())

#Loading Data
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)

In [55]:
#모델 생성
class CNN(nn.Module):
    def __init__ (self): #주요 층 정의
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1) #2개의 합성곱 층, 입력 1, 출력 32, 커널 3x3, 스트라이드 1
        self.conv2 = nn.Conv2d(32,64,3,1) #입력 32, 출력 64, 커널 3x3, 스트라이드 1
        self.dense1 = nn.Linear(12*12*64, 128) #2개의 밀집 층, 순서대로 입력 채널 수, 출력 채널 수 
        self.dense2 = nn.Linear(128, 10)

    def forward(self, x): #나머지 층 정의
        x = F.relu(self.conv1(x)) #init으로 정의
        x = F.relu(self.conv2(x)) 
        x = F.max_pool2d(x,2,2) #공간 해상도를 줄임
        x = F.dropout(x,0.25) #과적합 방지 드롭아웃
        x = x.view(-1,12*12*64) #탠서의 형태를 변형, -i인자는 나머지 차원을 자동으로 계산하라는 말
        x = F.relu(self.dense1(x)) 
        x = F.dropout(x,0.5)
        x = self.dense2(x)
        return F.log_softmax(x,dim=1) #소프트맥스 함수의 로그 버전을 계산함
        

In [56]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4%}'.format(epoch+1, num_epoches, i+1, total_step, loss.item()))

Epoch [1/15], Step [100/469], Loss: 31.3846%
Epoch [1/15], Step [200/469], Loss: 29.7113%
Epoch [1/15], Step [300/469], Loss: 16.6429%
Epoch [1/15], Step [400/469], Loss: 6.4815%
Epoch [2/15], Step [100/469], Loss: 5.3084%


KeyboardInterrupt: 